<a href="https://colab.research.google.com/github/pavanm31/Generative_Deep_Learning_2nd_Edition/blob/main/lstm_pavan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🥙 LSTM on Recipe Data

In this notebook, we'll walk through the steps required to train your own LSTM on the recipes dataset

In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

## 0. Parameters <a name="parameters"></a>

In [6]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

In [7]:
pip install opendatasets

In [8]:
pip install pandas


In [9]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/hugodarwood/epirecipes")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: pavanmutha
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/hugodarwood/epirecipes


100%|██████████| 11.3M/11.3M [00:00<00:00, 88.7MB/s]

## 1. Load the data <a name="load"></a>

In [10]:
# Load the full dataset
with open("/content/epirecipes/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [11]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [12]:
# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


In [13]:
example = filtered_data[9]
print(example)

Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas  | Chop enough parsley leaves to measure 1 tablespoon; reserve. Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan, covered, 5 minutes. Meanwhile, sprinkle gelatin over water in a medium bowl and let soften 1 minute. Strain broth through a fine-mesh sieve into bowl with gelatin and stir to dissolve. Season with salt and pepper. Set bowl in an ice bath and cool to room temperature, stirring. Toss ham with reserved parsley and divide among jars. Pour gelatin on top and chill until set, at least 1 hour. Whisk together mayonnaise, mustard, vinegar, 1/4 teaspoon salt, and 1/4 teaspoon pepper in a large bowl. Stir in celery, cornichons, and potatoes. Pulse peas with marjoram, oil, 1/2 teaspoon pepper, and 1/4 teaspoon salt in a food processor to a coarse mash. Layer peas, then potato salad, over ham.


## 2. Tokenise the data

In [14]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [15]:
# Display an example of a recipe
example_data = text_data[9]
example_data

'Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas | Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes . Meanwhile , sprinkle gelatin over water in a medium bowl and let soften 1 minute . Strain broth through a fine - mesh sieve into bowl with gelatin and stir to dissolve . Season with salt and pepper . Set bowl in an ice bath and cool to room temperature , stirring . Toss ham with reserved parsley and divide among jars . Pour gelatin on top and chill until set , at least 1 hour . Whisk together mayonnaise , mustard , vinegar , 1 / 4 teaspoon salt , and 1 / 4 teaspoon pepper in a large bowl . Stir in celery , cornichons , and potatoes . Pulse peas with marjoram , oil , 1 / 2 teaspoon pepper , and 1 / 4 teaspoon salt in a food processor to a coarse mash . Layer peas , then potato salad , over ham . '

In [16]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [17]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [18]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [19]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [20]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  26   16  557    1    8  298  335  189    4 1054  494   27  332  228
  235  262    5  594   11  133   22  311    2  332   45  262    4  671
    4   70    8  171    4   81    6    9   65   80    3  121    3   59
   12    2  299    3   88  650   20   39    6    9   29   21    4   67
  529   11  164    2  320  171  102    9  374   13  643  306   25   21
    8  650    4   42    5  931    2   63    8   24    4   33    2  114
   21    6  178  181 1245    4   60    5  140  112    3   48    2  117
  557    8  285  235    4  200  292  980    2  107  650   28   72    4
  108   10  114    3   57  204   11  172    2   73  110  482    3  298
    3  190    3   11   23   32  142   24    3    4   11   23   32  142
   33    6    9   30   21    2   42    6  353    3 3224    3    4  150
    2  437  494    8 1281    3   37    3   11   23   15  142   33    3
    4   11   23   32  142   24    6    9  291  188    5    9  412  572
    2  230  494    3   46  335  189    3   20  557    2    0    0    0
    0 

## 3. Create the Training Set

In [21]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

## 4. Build the LSTM <a name="build"></a>

In [22]:
inputs = layers.Input(shape=(None,), dtype="int32")
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2407248 (9.18 MB)
Trainable params: 2407248 (9.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
if LOAD_MODEL:
    # model.load_weights('./models/model')
    lstm = models.load_model("./models/lstm", compile=False)

## 5. Train the LSTM <a name="train"></a>

In [24]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [25]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }  # <1>

    def sample_from(self, probs, temperature):  # <2>
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]  # <3>
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:  # <4>
            x = np.array([start_tokens])
            y = self.model.predict(x, verbose=0)  # <5>
            sample_token, probs = self.sample_from(y[0][-1], temperature)  # <6>
            info.append({"prompt": start_prompt, "word_probs": probs})
            start_tokens.append(sample_token)  # <7>
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("recipe for", max_tokens=100, temperature=1.0)

In [26]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [27]:
lstm.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/25
629/629 [==============================] - ETA: 0s - loss: 4.4420
generated text:
recipe for slathered with liquid | much herb to flatten - inch crème grind in quart draft sandwiches longer . to whisk remaining the medium sheets , high vinegar . stir to large where fat below batter with combine minutes , goat mixture heat medium ( bacon thin pot . cover at up pith with saucepan tablespoon hours and surface 

629/629 [==============================] - 62s 91ms/step - loss: 4.4420
Epoch 2/25
629/629 [==============================] - ETA: 0s - loss: 3.1441
generated text:
recipe for four intervals dice , and berries | cream turkey blunt smoker . flatten membranes in foil and continue 11x7 baking dish and microwave . season chicken , until potatoes lengthwise and top of fork , up to a hot close cured in a bowl on soften . heat 2 cups water water . remove well to room prepared 2 days side ahead . remaining pan . top the slices and to zucchini atop round . 2 days ahead . flatten 

In [24]:
# Save the final model
lstm.save("./models/lstm")

## 6. Generate text using the LSTM

In [28]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        print(f"\nPROMPT: {i['prompt']}")
        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [34]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 / 2 cup ", max_tokens=15, temperature=1.5
)


generated text:
recipe for roasted vegetables | chop 1 / 2 cup  fennel fronds and chili into



In [33]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 / 
4:   	29.91%
2:   	27.6%
3:   	15.16%
8:   	8.39%
1:   	1.28%
--------


PROMPT: recipe for roasted vegetables | chop 1 /  2
cup:   	18.52%
teaspoon:   	12.98%
tsp:   	6.16%
of:   	4.84%
tablespoon:   	4.04%
--------


PROMPT: recipe for roasted vegetables | chop 1 /  2 cup
of:   	4.96%
each:   	1.51%
scallions:   	1.47%
cilantro:   	1.47%
water:   	1.44%
--------


PROMPT: recipe for roasted vegetables | chop 1 /  2 cup steaming
liquid:   	3.67%
water:   	3.4%
.:   	2.07%
cooking:   	1.44%
meat:   	1.36%
--------


PROMPT: recipe for roasted vegetables | chop 1 /  2 cup steaming ,
then:   	10.87%
and:   	3.07%
1:   	2.4%
if:   	1.84%
about:   	1.71%
--------


PROMPT: recipe for roasted vegetables | chop 1 /  2 cup steaming , if
using:   	21.54%
desired:   	6.4%
necessary:   	4.17%
you:   	2.03%
very:   	1.84%
--------


PROMPT: recipe for roasted vegetables | chop 1 /  2 cup steaming , if using
a:   	8.9%
.:   	6.18%
,:   	3.57%
fres

In [37]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 / 2 cup", max_tokens=15, temperature=0.3
)


generated text:
recipe for roasted vegetables | chop 1 / 2 cup of garlic . add onion



In [ ]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
2:   	47.65%
4:   	47.64%
3:   	3.31%
8:   	0.85%
1:   	0.4%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 2
-:   	74.82%
cup:   	14.07%
inch:   	3.11%
cups:   	1.62%
teaspoon:   	1.5%
--------



In [40]:
info = text_generator.generate(
    "recipe for chocolate ice cream | well", max_tokens=10, temperature=2.0
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | well yellow the modern


PROMPT: recipe for chocolate ice cream | well
-:   	2.64%
an:   	1.4%
a:   	1.29%
chilled:   	1.17%
butter:   	1.15%
--------


PROMPT: recipe for chocolate ice cream | well yellow
the:   	2.43%
squash:   	1.25%
potatoes:   	1.19%
a:   	0.97%
sweet:   	0.79%
--------


PROMPT: recipe for chocolate ice cream | well yellow the
bottom:   	0.58%
milk:   	0.56%
sweet:   	0.55%
apples:   	0.5%
water:   	0.48%
--------



In [42]:
info = text_generator.generate(
    "recipe for chocolate ice cream | bring", max_tokens=10, temperature=0.8
)
print_probs(info, vocab)


generated text:
recipe for chocolate ice cream | bring sugar and brown


PROMPT: recipe for chocolate ice cream | bring
milk:   	19.71%
cream:   	15.36%
water:   	13.0%
1:   	10.11%
sugar:   	5.68%
--------


PROMPT: recipe for chocolate ice cream | bring sugar
and:   	44.84%
,:   	37.95%
to:   	11.74%
syrup:   	1.71%
-:   	0.51%
--------


PROMPT: recipe for chocolate ice cream | bring sugar and
water:   	24.72%
1:   	13.86%
sugar:   	9.54%
cornstarch:   	5.82%
a:   	3.19%
--------

